In [ ]:
!pip install -U langchain langchain-community langchain-openai sentence-transformers faiss-cpu gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from google.colab import userdata
import gradio as gr
import os

In [ ]:
transcript_folder = "/content/drive/MyDrive/ServiceNow_Audio_Transcripts/"
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

intro_phrases = [
    "welcome", "in this video", "today we're", "this tutorial",
    "thank you for joining", "let’s get started"
]

def is_intro(text):
    return any(phrase in text.lower() for phrase in intro_phrases)

documents = []

for filename in os.listdir(transcript_folder):
    if filename.endswith(".txt"):
        with open(os.path.join(transcript_folder, filename), "r", encoding="utf-8") as f:
            text = f.read()
            chunks = splitter.split_text(text)
            for chunk in chunks:
                if not is_intro(chunk):
                    documents.append(Document(page_content=chunk, metadata={"source": filename}))

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
faiss_store = FAISS.from_documents(documents, embedding=embedding_model)

/tmp/ipython-input-14-1137632584.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
llm = OpenAI(
    temperature=0,
    openai_api_key=userdata.get("OPENAI_API_KEY")
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=faiss_store.as_retriever(),
    return_source_documents=True
)

In [ ]:
def answer_question(question):
    try:
        result = qa_chain(question)
        answer = result["result"]
        sources = []

        for doc in result["source_documents"]:
            snippet = doc.page_content.strip()
            snippet_preview = snippet[:400] + "..." if len(snippet) > 200 else snippet
            sources.append(f"📄 {doc.metadata['source']}\n🔎 {snippet_preview}")

        return answer, "\n\n".join(sources)

    except Exception as e:
        return f"Error: {str(e)}", f"Error: {str(e)}"

In [ ]:
code = '''from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain.docstore.document import Document
import gradio as gr
import os

openai_api_key = os.getenv("OPENAI_API_KEY")

documents = [Document(page_content="ServiceNow uses AI for incident classification.", metadata={"source": "intro.txt"})]

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
faiss_store = FAISS.from_documents(documents, embedding=embedding_model)

llm = OpenAI(temperature=0, openai_api_key=openai_api_key)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=faiss_store.as_retriever(),
    return_source_documents=True
)

def answer_question(question):
    result = qa_chain(question)
    answer = result["result"]
    sources = []
    for doc in result["source_documents"]:
        snippet = doc.page_content[:200].strip()
        sources.append(f"📄 {doc.metadata['source']}\n🔎 {snippet}")
    return answer, "\\n\\n".join(sources)

gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(placeholder="Ask a ServiceNow question..."),
    outputs=[gr.Textbox(label="Answer"), gr.Textbox(label="Sources")],
    title="ServiceNow QA Bot",
    description="Ask questions about ServiceNow based on transcripts."
).launch()
'''

with open("app.py", "w") as f:
    f.write(code)

In [ ]:
requirements = '''gradio
langchain
langchain-openai
langchain-community
sentence-transformers
faiss-cpu
'''

with open("requirements.txt", "w") as f:
    f.write(requirements)

In [ ]:
from google.colab import files
files.download("app.py")
files.download("requirements.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import gradio as gr

interface = gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(lines=2, placeholder="Ask me about a ServiceNow topic..."),
    outputs=[
        gr.Textbox(label="Answer"),
        gr.Textbox(label="Sources and Snippets")
    ],
    title="ServiceNow QA Assistant",
    description="Ask questions about the content in your YouTube transcripts."
)

interface.launch(share=True, debug=True, inline=False, show_error=True, prevent_thread_lock=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2f4f3bf9b81474fc88.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://2f4f3bf9b81474fc88.gradio.live
